In [1]:
from pathlib import Path
import sys
import yaml
import time

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

DIR = Path('..')
sys.path.append(str(DIR))
from SCanalyzer import SCanalyzer
from SCanalyzer.busSim import BusSim, Config
from SCanalyzer.busSim.manager import managerFactory

%matplotlib inline

In [2]:
def get_yelp_api_key():
    with open("../config.yml", 'r') as yml:
        cfg = yaml.safe_load(yml)
        return cfg["yelp"]["api_key"]

def plot_background():
    background_path = "../data/plot/background/"
    city = gpd.read_file(background_path + "madison-meter-shp")
    lakes = gpd.read_file(background_path + "water-meter-shp")
    street = gpd.read_file(background_path + "street-meter-shp")
    # city = gpd.read_file(background_path + "madison-shp")
    # lakes = gpd.read_file(background_path + "water-shp")
    # street = gpd.read_file(background_path + "street-shp")

    ax = city.plot(color="lightgray", alpha=.2, figsize=(12, 12), zorder=2)
    lakes.plot(color="lightblue", ax=ax, zorder=1, alpha=0.8)
    street.plot(color="darkgray", alpha = .5, ax=ax, zorder=3)
    return ax

In [3]:
sc = SCanalyzer("../data/mmt_gtfs.zip")
config = Config(day="monday", elapse_time="00:30:00", interval="10:00:00", max_walking_min=10) # run sim on Monday's schedule; run over a 30 min window for every hour; limit max walking to 10 min
census_gdf = sc.load_census() # load the census data of the region
config.set_starts(centroids=census_gdf) # set the starting points to be the centers of each census tracks
result_gdf = sc.search(config) # run Sim
services_gdf = sc.load_yelp(api_key=get_yelp_api_key()) # load service data from yelp
sc.add_service_metrics(result_gdf, services_gdf) # combine Sim result with service data
sc.add_demographic_metrics(result_gdf, census_gdf) # combine Sim result with census data
result_gdf

,geometry,start_time,map_identifier,banks,clinics,dentists,hospitals,supermarket,Tot Pop,cars per capita
0,POINT (-89.66743836656362 43.18492511575211),0:0:00,search-result-0-0!0,0.0,0.0,0.0,0.0,0.0,1844.0,0.404013
1,POINT (-89.55685462121274 42.99395548763492),0:0:00,search-result-0-0!1,0.0,0.0,0.0,0.0,0.0,1802.0,0.314095
2,POINT (-89.5513701948981 43.03058973687484),0:0:00,search-result-0-0!2,0.0,0.0,0.0,0.0,0.0,3575.0,0.439720
3,POINT (-89.52466340686239 42.97083580368362),0:0:00,search-result-0-0!3,0.0,0.0,0.0,0.0,0.0,4715.0,0.386002
4,POINT (-89.48838054600107 43.0984346448524),0:0:00,search-result-0-0!4,0.0,0.0,0.0,0.0,0.0,2256.0,0.475177
...,...,...,...,...,...,...,...,...,...,...
829,POINT (-89.26709580690887 43.02062363239731),20:0:00,search-result-0-1200!273,0.0,0.0,0.0,0.0,0.0,1600.0,0.333125
830,POINT (-89.254316326777 43.17947047459754),20:0:00,search-result-0-1200!274,0.0,0.0,0.0,0.0,0.0,2205.0,0.451247
831,POINT (-89.21923375584366 43.19099896987731),20:0:00,search-result-0-1200!275,0.0,0.0,0.0,0.0,0.0,1452.0,0.376033
832,POINT (-89.23107883723563 43.055505101709),20:0:00,search-result-0-1200!276,0.0,0.0,0.0,0.0,0.0,450.0,0.537778


In [4]:
result_gdf[result_gdf["banks"] != 0]

,geometry,start_time,map_identifier,banks,clinics,dentists,hospitals,supermarket,Tot Pop,cars per capita
287,POINT (-89.43742096467113 43.02934177489417),10:0:00,search-result-0-600!9,2.0,1.0,0.0,0.0,0.0,656.0,0.402439
288,POINT (-89.40843688667083 43.05988040176729),10:0:00,search-result-0-600!10,21.0,30.0,9.0,24.0,7.0,1352.0,0.418639
289,POINT (-89.39366773822951 43.0754185599703),10:0:00,search-result-0-600!11,32.0,65.0,33.0,33.0,14.0,1746.0,0.369416
290,POINT (-89.38811120771256 43.14381292339061),10:0:00,search-result-0-600!12,2.0,4.0,4.0,0.0,0.0,1035.0,0.427053
291,POINT (-89.38622626422388 43.07820878294773),10:0:00,search-result-0-600!13,35.0,54.0,24.0,30.0,12.0,1788.0,0.526286
...,...,...,...,...,...,...,...,...,...,...
820,POINT (-89.35708405080736 43.04670969714362),20:0:00,search-result-0-1200!264,7.0,2.0,1.0,1.0,1.0,570.0,0.507018
822,POINT (-89.34501637847924 43.09884323175058),20:0:00,search-result-0-1200!266,26.0,20.0,9.0,8.0,7.0,2088.0,0.472222
824,POINT (-89.33648264423933 43.0967663910675),20:0:00,search-result-0-1200!268,14.0,17.0,9.0,3.0,3.0,1317.0,0.504935
826,POINT (-89.32712297155682 43.1086703403175),20:0:00,search-result-0-1200!270,28.0,21.0,16.0,6.0,5.0,1038.0,0.421002


# route remove study

In [ ]:
gtfo = Gtfo("../data/mmt_gtfs.zip")
census_gdf = gtfo.load_census()
services_gdf = gtfo.load_yelp(api_key=get_yelp_api_key())

routes = [ 2,  4,  5,  6,  7,  8, 10, 11, 12, 13, 15, 16, 17, 18, 20, 21, 22,
       23, 26, 27, 30, 31, 32, 33, 36, 39, 40, 44, 48, 49, 50, 51, 52, 55,
       67, 70, 72, 73, 75, 59, 63, 68, 78, 80, 84, 81, 82]
for route in routes:
    print(route)
    gtfo.set_batch_label(f"{route}-rm")
    config = Config(day="monday", elapse_time="00:30:00", interval="00:10:00", max_walking_min=10, route_remove=[route])
    config.set_starts(centroids=census_gdf)

    perf_df = pd.DataFrame(
            columns=["geometry", "start_time", "search_time", "add_service_time", "add_census_time"])
    result_gdf = gtfo.search(config, perf_df)
    gtfo.add_service_metrics(result_gdf, services_gdf, perf_df)
    gtfo.add_demographic_metrics(result_gdf, census_gdf, perf_df)
    result_gdf.to_csv(f"../out/result/result{route}.csv", index=False)
    perf_df.to_csv(f"../out/perf/perf{route}.csv", index=False)

In [ ]:
perf = pd.read_csv("../out/perf/perf80.csv")
perf["search_time"].sum(), perf["add_service_time"].sum(), perf["add_census_time"].sum(),

In [6]:
sc = SCanalyzer("../data/mmt_gtfs.zip")
census_gdf = sc.load_census()
config = Config(day="monday", elapse_time="00:30:00", interval="10:00:00", max_walking_min=10)
# config.set_starts(points=[(43.073691, -89.387407), (43.073691, -89.387407)])
config.set_starts(centroids=census_gdf)
t0 = time.time()
result_gdf = sc.search(config)
t1 = time.time()
print(t1-t0)

services_gdf = sc.load_yelp(api_key=get_yelp_api_key())
sc.add_service_metrics(result_gdf, services_gdf)
t2 = time.time()
print(t2-t1)

sc.add_demographic_metrics(result_gdf, census_gdf) # SLOW
t3 = time.time()
print(t3-t2)
result_gdf["banks"].sum() #7263.0

14.490088939666748
6.0011491775512695
3.6119019985198975


4433.0

# Visualize simulation

In [ ]:
gtfo = Gtfo("../data/mmt_gtfs.zip")
ax = plot_background()
gtfo.set_batch_label(f"80-rm")
example_gdf = gtfo.load_result_map(map_identifier="search-result-0-600!10")
example_gdf.plot(color="#ffbfba", ax=ax)

# AWS test

In [2]:
from SCanalyzer.busSim.manager import managerFactory
aws = managerFactory.create("aws", gtfs_path=None, city_path=None, out_path=None)

creating s3 bucket
creating IAM role


In [3]:
aws.clean_up()

cleaning up aws resources
cleaning up s3
cleaning up lambda


In [ ]:
import boto3
aws_lambda = boto3.client('lambda', region_name='ap-northeast-1')
with open("tmp.zip", 'rb') as f:
    response = aws_lambda.create_function(
        Code={
            'ZipFile': f.read()
        },
        Description='BusSim handler',
        FunctionName="test",
        Handler='lambda_function.lambda_handler',
        MemorySize=512,
        Publish=True,
        Role="arn:aws:iam::347664766527:role/s3rwRole",
        Runtime='python3.8',
        Timeout=900,
        TracingConfig={
            'Mode': 'Active',
        },
    )